# Modelos *Ensemble*

Una de las últimas tendencias dentro de lo que serían los modelos de inteligencia artificial viene a resumirse como "el conocimiento del conjunto o la multitud". Lo que viene a definir estar frase, un tanto popular, es el uso de multitud de modelos denominados "débiles" con el fin de combinar el conocimiento extraido en un modelo fuerte. Este mismo hecho ya lo hemos visto en una aproximación anterior comom es el *Random Forest*, en donde este se desarrolla en base a la creación de multiples *Decision Trees* mucho más simples pero cuyo resultado excede a cualquiera de ellos. Esto modelos surgidos de está manera reciben el nombre de modelos *Ensemble*.

Sin embargo, estos modelos pueden no limitarse unicamente a los arboles de decisión si no que pueden ser creados mediante el uso combinado de varias técnicas como pueden ser el K-NN, SVM, etc.

En la presente unidad vamos a explorar el como se comportan varias de estas maneras de combinación de modelos y las extategias más habituales para crear los modelos.

## Descargando los datos 

Vamos a volver a cargar el problema que venimos usando como banco de pruebas de clasificaicón, es decir, el de **¿Roca o Mina?**

In [ ]:
import os.path
import pandas as pd

def load_data(name, url):
    #check if the data is already downloaded, otherwise download it
    if not os.path.isfile(name):
        print(f'Downloading'.ljust(75,'.'), end='', flush=True)
        import urllib.request
        urllib.request.urlretrieve(url,name)
        print(f"Done!")
    return pd.read_csv(name, delimiter=',', header=None)

file_name = '_data_/sonar.all_data'
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data'
data = load_data(file_name, url)
    
data['Mina'] = (data[60]=='M')
data.head(5)

El siguiente paso en la preparación sera dividir el conjunto de datos en entrenamieno y test de tal manera que podemos comparar las diferentes alternativas, que vamos a explorar. Para realizar este proceso lo más sencillo es extraer las matrices y dividir los conjuntos medianate el uso de la función de **scikit-learn** denominada **train_test_split**. Este procedimiento en el cual se reserva un conjunto de los datos para el test posterior recibe el nombre de *hold-out*. Un rango habitual una división habitual para esto es un 70:30 pero debido a que tenemos pocos patrones vamos a reducir ese montante a solo un 10% del total.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

#Recoger las 60 primeras mediciones y convertirlas a un Numpy
#no tienen nombre así que accedemos según la posición
inputs = (data.iloc[:,0:60]).to_numpy()
#Convertir la salida a un formato numérico y un numpy
outputs = (data['Mina']).to_numpy().astype('int')

#Crear los conjuntos de entrenamiento y test
train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(inputs, outputs, test_size=0.1, stratify=outputs)

print(f"Train Patterns{train_inputs.shape} -> {train_outputs.shape}")
print(f"Test Patterns{test_inputs.shape} -> {test_outputs.shape}")

Un punto importante a descatar es que la división se ha hecho con un esquema *strastified*. Este parámetro lo que garantiza es que las clases presentes están representadas en el conjunto de test en la misma proporción que en el conjunto general. 

![Ejemplo de Stratified KFold](img/StratifiedKFold.png)

En el ejemplo sobre estas líneas se puede ver como en tre divisiones diferentes, el sistema ha escogido diferentes patrones para el entrenamiento y el test, pero siempre ha guardado la proporción. Para guardar esa proporción en nuestro ejemplo bastará con pasarle a la función **train_test_split** el parámetro **stratify** identificando en un array las clases que queremos que se contemplen. Ese array en nuestro caso no será otro que la variable outputs.

## Estableciendo la línea base
Una vez tenemos los datos preparados vamos a establecer una línea base con algunas de las técnicas que ya se han visto durante el curso. Para ello, en lugar de variables para cada uno de ellos, vamos a usar un `Dictionary` de Python que nos permitira rener una sola y referirnos por nombre al modelo en su interior.

In [ ]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB 

clfs = { 'SVM': svm.SVC(probability=True), 
         'LR': LogisticRegression(),
         'DT': DecisionTreeClassifier(max_depth=4),
         'NB':GaussianNB()}
base_models = ['SVM', 'LR','DT','NB']

for key in clfs.keys():
    clfs[key].fit(train_inputs, train_outputs)
    acc = clfs[key].score(test_inputs, test_outputs)
    print(f"{key}: {(acc*100):.4f}%")

### Ejercicio:
Ajuste las técnicas anteriores variando los parámetros buscando mejorar su respuesta individual ante el problema. Alternativamente, añada otras técnicas de classificaicón, por ejemplo, K-NN.

Como se puede ver hay métodos que dan un buen resultado de ajuste desde el comienzo como es el caso del SVM para este problema, si bien otros es posible que necesitaran algo más de trabajo. Todos estos modelos es lo que se consideran modelos simples o *'weak models'*, en las técnicas ensemble lo que se busca es combinar varios de estos modelos con el fin de mejorar el rendimiento global. 

## Combinando Modelos
A la hora de combinar los modelos existen diferentes estrategias según la tarea del modelo, es dicer, si estamos clasificando o haciendo una regresión. En este caso nos vamos a centrar en la classificación, si bien para la regresión sería similar, habría que tener en cuenta el caracter contínuo de los valores a la hora de combinar las salidas.

En cuanto a la combinación de la classificación, existen principalmente dos maneras de combinar las salidas de varios clasificadores. Estas combinaciones reciben el nombre de *Majority/Hard Voting* y *Soft Voting* 

### Hard Voting
Como su nombre indica se basan en seleccionar la opción más votada entre las predichas entre los diferentes modelos. Simplemente con sumar las salidas de las diferentes categorias y establecer un umbral seremos quienes de resolver nuestro problema enfocado desde diferentes puntos de vista. Para poder ejecutarlo simplemente haremos uso de la siguiente función

In [ ]:
from sklearn.ensemble import VotingClassifier

#Define the ensemble with the classifier previously used
clfs['Ensemble (Hard Voting)'] = VotingClassifier (estimators = [(name,clfs[name]) for name in base_models], 
                                                   n_jobs=-1)
clfs['Ensemble (Hard Voting)'].fit(train_inputs, train_outputs)

for key in clfs.keys():
    acc = clfs[key].score(test_inputs, test_outputs)
    print(f"{key}: {(acc*100):.4f}%")

Como se puede, si bien no mejora al mejor de los modelos que lo componen, esto es una única ejecución y el problema es que todos los modelos pesan lo mismo en la decisión entre clases. Para solucionar este problema o bien se puede incrementar el número de modelos, o bien hacer que las respuestas no pesen lo mismo en las decisiones. Este último punto es el que exploraremos en primer lugar en el siguiente apartado

### Soft Voting o Vota Mayoritario ponderado
En este caso lo que se hace es asignarle un peso a las decisiones que se toman por parte de los modelos ya que en algunos se confia más que en otros. Para la asignación de la importancia lo que se toma es la probabilidad de las asignación a una clase determinada y se ponderá según un conjunto de pesos. Imaginemos que a cada uno de los classificadores le asignamos el mismo peso, es decir {1,1,1}.

|Classificador	     |Mina	        |Roca          |
| :------------- | :----------: | -----------: |
|SVM         	 |w1 * 0.8	    |w1 * 0.2      |	
|LR         	 |w2 * 0.1	    |w2 * 0.9      |	
|NB         	 |w3 * 0.1	    |w3 * 0.9      |
|Soft Voting      |0.33	        |0.67          |	

Así la clase seleccionada sería la de Roca ya que todos pesan lo mismo pero si sabes que uno de los modelos es más fiable lo que se puede hacer es inclementar su peso como se ve a continuación. Esto le dará mayor robusted como se puede ver en los resultados

In [ ]:

clfs['Ensemble (Soft Voting)'] = VotingClassifier (estimators = [(name,clfs[name]) for name in base_models], 
                                                   n_jobs=-1, voting='soft',weights=[1,1,1,3])
clfs['Ensemble (Soft Voting)'].fit(train_inputs, train_outputs)

for key in clfs.keys():
    acc = clfs[key].score(test_inputs,test_outputs)
    print(f"{key}: {(acc*100):.4f}%")

Como se puede ver los resultados son mejores cuando se combinan varios modelos que dan buenos resultados. De hecho este procedimiento es la base de otras técnicas que hemos visto con anteioridad como son los *Random Forest*. Los modelos a usar son la otra de las claves para la creación _ensemble_, en la siguiente sección veremos las estrategias más habituales para la creación de los modelos.

### Stacking de modelos

Este es el caso más avanzado ya que se crea un modelo sobre los modelos. Este no está limitado a que se utilice un tipo de elemento u otro, este metaclasificador o metaregresor (según el problema) puede ser desde un SVM, una Red de Neuronas Artificiales, etc.

Vemos a continuación un ejemplo de este:


In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier


estimators = [
    ('SVM', svm.SVC(probability=True)), 
    ('LR', LogisticRegression()),
    ('DT', DecisionTreeClassifier(max_depth=4)),
    ('NB',GaussianNB())
]
clf['Ensemble (Stacking)'] = StackingClassifier(
    estimators=estimators, 
    final_estimator=MLPClassifier(), 
    cv=5,
    n_jobs=-1
    )

clf['Ensemble (Stacking)'].fit(train_inputs, train_outputs)

for key in clfs.keys():
    acc = clfs[key].score(test_inputs,test_outputs)
    print(f"{key}: {(acc*100):.4f}%")


Dentro de este tipo de modelos de metaclasifición los parámetros principales son:

* **estimators**, el cual debe de contener una lista con los clasificadores que se quieren utilizar.
* **final_estimator**, que determina el modelo que recibira como entradas las salidas de los `estimators`indicados anteriormente.
* **cv**, este parámetro asume que por defecto un valor de 5 con el que se entrenan primero los estimadores iniciales, y porteriormente el "estimador final" en caso de que este parámetro se le pase el valor `'prefit'` se asume que los estimadores estan ya ajustados y los datos se usan sólo para ajustar el "estimador final".
* **stack_method**, segun el tipo de medida puede ser interesarnte usar el método predict o el predict_proba de los estimadores. En todo caso, el valor por defecto es `auto` que identificará el más adecuado en función del tipo de estimadores. 


## Creación de modelos

Uno de los elementos clave que no hemos abordado todavía en la creación de los modelos que iran en el ensemble. Hasta ahora todos los modelos se habían creado sobre el mismo conjunto de entrenamiento para cada uno de los modelos sencillos que habíamos creado. Sin embargo, esto no es la práctica habitual si no que el conjunto de patrones de entrada se suele repartir en conjuntos más pequeños con los que entrenar una o varias técnicas con el fin de reducir el coste computacional y minimizar los modelos.
Esa estrategia de reparto toma habitualmente 2 formas principales conocidas como *Bagging* y *Boosting*.

### Bagging o boostrap aggregation
La primera de las técnicas que vamos a ver el es Bagging o selección con remplazo. Esta técnica se basa en desarrollar diferentes modelos los cuales son entrenados sobre un subconjunto del conjunto de entrenamiento el cual se extrae de manera aleatoria. Esta extracción de los ejemplos se realiza con remplazo, es decir, una vez que un ejemplo ha sido seleccionado de las posibilidades se coloca nuevamente entre las posibilidades para los subconjuntos de las otros modelos, o lo que es lo mismo se crean conjuntos no disjuntos de ejemplos.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Ensemble_Bagging.svg/440px-Ensemble_Bagging.svg.png" alt="Ejemplo de Bagging" style="width:500px;"/>

Una técnica muy conocida que usa está aproximación son los RandomForest que construyen los arboles de decision que los componen de esta manera.

A continuación se puede ver a continuación se determina el número máximo de modelos así como su complejidad e internamente el modelo de RandomForest realiza esa división en subconjuntos.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clfs['RF'] = RandomForestClassifier(n_estimators=20, max_depth=None,
                                    min_samples_split=10, n_jobs=-1)

clfs['RF'].fit(train_inputs, train_outputs)
for key in clfs.keys():
    acc = clfs[key].score(test_inputs,test_outputs)
    print(f"{key}: {(acc*100):.4f}%")

Si se quiere, se puede usar cualquier classificador como base de un *Bagging*. Simplemente será neceario el uso de la clase [BaggingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html?highlight=classifier#sklearn.ensemble.BaggingClassifier)

### Boosting
La otra gran familia de técnica de ensemble es lo que se conoce como *Boosting*. Estas se basan en seleccionar un conjun subconjunto de patrones del conjunto original. Este proceso se hace de manera secuencial y sin remplazo de tal manera que cada vez vamos teniendo clasificadores más específicos que se concentran en aquellos ejemplos menos frecuentes o que han sido clasificados de manera incorrecta en un paso anterior.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Ensemble_Boosting.svg/1920px-Ensemble_Boosting.svg.png" alt="Ejemplos de Boosting" style="width:700px;"/>

La idea detrás de esta aproximación es que no todos los modelos tengan que tener una base común como en el caso del *Bagging*. Existen multitud de ejemmplos que utilizan este tipo de técnica como, por ejemplo, El [Ada Boost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html?highlight=classifier#sklearn.ensemble.AdaBoostClassifier) o el [Gracient Tree Boosting](https://sklearn.org/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier), cuyo ejemplo se puede ver a continuación

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

clfs['Ada'] = AdaBoostClassifier(n_estimators=30)
clfs['Ada'].fit(train_inputs, train_outputs)

clfs['GTB'] = GradientBoostingClassifier(n_estimators=30, learning_rate=1.0, max_depth=2, random_state=0)
clfs['GTB'].fit(train_inputs, train_outputs)

for key in clfs.keys():
    acc = clfs[key].score(test_inputs,test_outputs)
    print(f"{key}: {(acc*100):.4f}%")


### Ejercicio
Todos los modelos que hemos creado llevan las señales tal cual, intere realizar algún tipo de preprocesado como una PCA de los datos y construya su propio ensemble en base a alguno de los modelos vistos en clase u otros.